In [1]:
#!/usr/bin/env python3
# dhs_orphanhood_under18.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

# ------------------------------------------------------------------
# 1. SETTINGS
# ------------------------------------------------------------------
FILE_PATH = os.path.join("data", "RWPR81FL.DTA")

# Variable Map
VAR_WT       = "hv005"       # Weight
VAR_AGE      = "hv105"       # Age
VAR_RESIDENT = "hv102"       # Usual Resident (1=Yes) -> De jure
VAR_REG      = "hv024"       # Region
VAR_DIST     = "shdistrict"  # District
VAR_MOM_ALIVE= "hv111"       # Mother Alive (0=No, 1=Yes, 8=DK)
VAR_DAD_ALIVE= "hv113"       # Father Alive (0=No, 1=Yes, 8=DK)

# Mapping for Eastern Province (Region 5)
DIST_MAP  = {
    51: 'Rwamagana', 
    52: 'Nyagatare', 
    53: 'Gatsibo',
    54: 'Kayonza',
    55: 'Kirehe',
    56: 'Ngoma',
    57: 'Bugesera'
}

# ------------------------------------------------------------------
# 2. CALCULATION (Modified for 1 Decimal)
# ------------------------------------------------------------------
def get_orphan_pct(df):
    """
    Calculates % of children with one or both parents dead.
    Returns float rounded to 1 decimal place.
    """
    if df.empty or df['w'].sum() == 0:
        return 0.0
    val = np.average(df['is_orphan'], weights=df['w']) * 100
    # CHANGED: Round to 1 decimal instead of converting to int
    return round(val, 1)

# ------------------------------------------------------------------
# 3. ANALYSIS PIPELINE
# ------------------------------------------------------------------
if __name__ == "__main__":
    if not os.path.exists(FILE_PATH):
        print(f"❌ Error: {FILE_PATH} not found.")
        exit()

    print("Loading Data...")
    df = pd.read_stata(FILE_PATH, convert_categoricals=False)
    df.columns = df.columns.str.lower()

    # --- FILTERS ---
    # 1. De jure Population (Usual Residents)
    df = df[df[VAR_RESIDENT] == 1]

    # 2. Children Under 18
    df = df[df[VAR_AGE] < 18]

    # 3. Weights
    df["w"] = df[VAR_WT] / 1000000.0

    # --- DEFINE INDICATOR ---
    # Condition: Mother Dead (0) OR Father Dead (0)
    # Codes: 0=Dead, 1=Alive, 8=DK. We count 0 only.
    df["mother_dead"] = (df[VAR_MOM_ALIVE] == 0)
    df["father_dead"] = (df[VAR_DAD_ALIVE] == 0)
    
    # Combined: One or both are dead
    df["is_orphan"] = (df["mother_dead"] | df["father_dead"]).astype(int)

    # --- CALCULATE ---
    results = []

    # A. District Level (Eastern Province - Region 5)
    EASTERN_REGION_CODE = 5
    df_eastern = df[df[VAR_REG] == EASTERN_REGION_CODE].copy()
    
    # Check for correct district column name (fallback safety)
    dist_col = VAR_DIST
    if VAR_DIST not in df_eastern.columns:
        if 'sdistrict' in df_eastern.columns:
            dist_col = 'sdistrict'
    
    if dist_col in df_eastern.columns:
        dist_stats = df_eastern.groupby(dist_col).apply(
            lambda x: pd.Series({"% Orphanhood": get_orphan_pct(x)})
        )
        dist_stats = dist_stats.rename(index=DIST_MAP)
        # Filter to keep only the districts in our map
        dist_stats = dist_stats[dist_stats.index.isin(DIST_MAP.values())]
        results.append(dist_stats)

    # B. Province Level
    results.append(pd.DataFrame(
        {"% Orphanhood": [get_orphan_pct(df_eastern)]},
        index=["Eastern Province"]
    ))    

    # C. National Level
    results.append(pd.DataFrame(
        {"% Orphanhood": [get_orphan_pct(df)]},
        index=["Rwanda"]
    ))

    # Combine
    final_df = pd.concat(results)
    
    print("\n--- Children <18 with One or Both Parents Dead (De jure) ---")
    print(final_df)

    # --- JSON OUTPUT ---
    json_filename = "eastern_Orphanhood_Under18.json"
    
    output_dict = {
        "indicator": "Percentage of Children <18 with One or Both Parents Dead",
        "unit": "Percentage (%)",
        "population_type": "De jure (Usual Residents)",
        "data": final_df.to_dict(orient="index") # Groups by location name
    }
    
    with open(json_filename, "w") as f:
        json.dump(output_dict, f, indent=4)
        
    print(f"✅ JSON saved: {json_filename}")

    # ------------------------------------------------------------------
    # 4. PLOTTING (WITH COLORS & DECIMALS)
    # ------------------------------------------------------------------
    
    # Assign Colors based on the Label Name
    bar_colors = []
    for label in final_df.index:
        if label == "Rwanda":
            bar_colors.append("#2E8B57")  # Green
        elif label == "Eastern Province":
            bar_colors.append("#FFD700")  # Yellow (Gold)
        else:
            bar_colors.append("#CD5C5C")  # Red (Districts)

    ax = final_df.plot(kind="bar", legend=False, color=bar_colors, figsize=(10, 7),
                       width=0.6, edgecolor="white")

    plt.title("Percentage of Children <18 with One or Both Parents Dead\n(De jure Population)", 
              fontsize=16, fontweight="bold")
    plt.xticks(rotation=0, fontsize=12)
    plt.grid(axis="y", ls="--", alpha=0.3)
    
    ax.yaxis.set_visible(False)
    for s in ["top", "right", "left"]: 
        ax.spines[s].set_visible(False)

    # Labels (UPDATED FOR DECIMALS)
    for c in ax.containers:
        # Format string changed from int() to :.1f (one decimal)
        labels = [f"{v.get_height():.1f}%" for v in c]
        ax.bar_label(c, labels=labels, label_type="edge", padding=3,
                     fontsize=12, fontweight="bold", color="black")

    plt.tight_layout()
    plt.savefig("eastern_Orphanhood_Under18.png", dpi=300)
    plt.close()
    print("✅ Graph saved: eastern_Orphanhood_Under18.png")

Loading Data...


/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_1552/1889681628.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["w"] = df[VAR_WT] / 1000000.0
/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_1552/1889681628.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["mother_dead"] = (df[VAR_MOM_ALIVE] == 0)
/var/folders/jr/x6fm65cx0hlblk09pm9kft3h0000gn/T/ipykernel_1552/1889681628.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 


--- Children <18 with One or Both Parents Dead (De jure) ---
                  % Orphanhood
Rwamagana                  5.6
Nyagatare                  7.7
Gatsibo                    7.0
Kayonza                    6.0
Kirehe                     7.4
Ngoma                      7.1
Bugesera                   4.9
Eastern Province           6.6
Rwanda                     6.8
✅ JSON saved: eastern_Orphanhood_Under18.json
✅ Graph saved: eastern_Orphanhood_Under18.png
